# Imports

In [1]:
import pandas as pd
import numpy as np
import random
from faker import Faker

# Dados Ficticios

## 1. Meta Ads

In [2]:
fake = Faker('pt_BR')
random.seed(42)
np.random.seed(42)

# Listas fictícias
campaign_names = ['Camisetas Filmes', 'Camisetas Series', 'Blusas Inverno', 'Acessorios']
ad_set_names = ['Homens 18-24', 'Mulheres 18-24', 'Homens 24-30', 'Mulheres 24-30']
ad_names = ['Imagem 1', 'Video 1', 'Carrossel 1', 'Imagem 2', 'Video 2', 'Imagem 3', 'Video 3', 'Carrossel 2']

# Função para gerar dados
def generate_meta_ads_data(n=100):

    meta_data = []

    for _ in range(n):
        campaign = np.random.choice(campaign_names, p=[0.4, 0.3, 0.2, 0.1] )
        ad_set = np.random.choice(ad_set_names, p=[0.4, 0.3, 0.15, 0.15])
        ad = random.choice(ad_names)
        impressions = random.randint(1000, 20000)
        clicks = random.randint(50, impressions // 10)
        spend = round(random.uniform(50, 1500), 2)
        ctr = round(clicks / impressions * 100, 2)
        cpc = round(spend / clicks, 2)
        conversions = random.randint(0, clicks)
        cpa = round(spend / conversions, 2) if conversions > 0 else 0
        purchase_value = round(random.uniform(0.5, 2.5) * conversions * 50, 2)
        roas = round(purchase_value / spend, 2) if spend > 0 else 0

        meta_data.append({
            'campanha': campaign,
            'conjunto_anuncio': ad_set,
            'anuncio': ad,
            'impressoes': impressions,
            'cliques': clicks,
            'gasto_total': spend,
            'ctr (%)': ctr,
            'cpc (R$)': cpc,
            'conversões': conversions,
            'cpa (R$)': cpa,
            'Receita': purchase_value,
            'roas': roas,
            'data': fake.date_between(start_date='-30d', end_date='today')
        })

    return pd.DataFrame(meta_data)



In [3]:
# meta_df já vira um DataFrame direto:
meta_df = generate_meta_ads_data(n=1000)
meta_df['data'] = pd.to_datetime(meta_df['data'])
meta_df['dia_da_semana'] = meta_df['data'].dt.day_name(locale='pt_BR')

## 2. CRM Leads

In [4]:
# Configuração
fake = Faker('pt_BR')
random.seed(42)
np.random.seed(42)

# Simular ad_ids e campanhas
ad_ids = ['Imagem 1', 'Video 1', 'Carrossel 1', 'Imagem 2', 'Video 2', 'Imagem 3', 'Video 3', 'Carrossel 2']
campaign_names = ['Camisetas Filmes', 'Camisetas Series', 'Blusas Inverno', 'Acessorios']
ad_id_to_campaign = {ad_id: random.choice(campaign_names) for ad_id in ad_ids}

# Etapas do funil e canais
funnel_stages = ["Visita", "Carrinho", "Checkout", "Comprou"]
channel_name = ["Instagram", "Facebook", "Google", "Tráfego Direto", "Email"]

def generate_crm_data(n=100):
    crm_data = []

    for i in range(n):
        lead_id = f"lead_{i+1}"
        nome = fake.name()
        nome_email = nome.lower().replace(' ', '.').replace('á','a').replace('ã','a').replace('â','a')\
                    .replace('é','e').replace('ê','e').replace('í','i').replace('ó','o')\
                    .replace('ô','o').replace('õ','o').replace('ú','u').replace('ç','c')
        email = f"{nome_email}@email.com"
        telefone = fake.phone_number()
        data_captura = fake.date_between(start_date='-60d', end_date='today')
        
        ad_clicked = random.choice(ad_ids)
        campanha_origem = ad_id_to_campaign[ad_clicked]
        #campanha_origem = np.random.choice(campaign_names, p=[0.4, 0.3, 0.2, 0.1])
        canal_origem = np.random.choice(channel_name, p=[0.3, 0.25, 0.25, 0.15, 0.05])

        # Sorteia a etapa do funil
        etapa = np.random.choice(funnel_stages, p=[0.4, 0.3, 0.2, 0.1])
        
        # Define status baseado na etapa
        if etapa == "Comprou":
            status = np.random.choice(["Ganhou", "Em negociação"], p=[0.85, 0.15])
        elif etapa in ["Carrinho", "Checkout"]:
            status = np.random.choice(["Em negociação", "Perdido"], p=[0.6, 0.4])
        else:  # Visita
            status = np.random.choice(["Novo", "Perdido"], p=[0.7, 0.3])
        
        # Ajusta a pontuação baseado na etapa (leads que compraram ou estão em checkout têm nota maior)
        if etapa == "Comprou":
            pontuacao = random.randint(80, 100)
        elif etapa == "Checkout":
            pontuacao = random.randint(60, 90)
        elif etapa == "Carrinho":
            pontuacao = random.randint(40, 80)
        else:  # Visita
            pontuacao = random.randint(10, 60)

        ultima_interacao = data_captura + pd.Timedelta(days=random.randint(0, 10))

        crm_data.append({
            "lead_id": lead_id,
            "nome": nome,
            "email": email,
            "telefone": telefone,
            "data_captura": str(data_captura),
            "campanha_origem": campanha_origem,
            "ad_clicked": ad_clicked,
            "canal_origem": canal_origem,
            "etapa_funil": etapa,
            "ultima_interacao": str(ultima_interacao),
            "status": status,
            "pontuacao": pontuacao
        })

    return pd.DataFrame(crm_data)

In [5]:
# meta_df já vira um DataFrame direto:
crm_df = generate_crm_data(n=300)

## 3. Vendas Site

In [6]:
# Configuração
fake = Faker('pt_BR')
random.seed(42)
np.random.seed(42)

# Relação entre campanha e produto
campanha_produto_relacao = {
    'Camisetas Filmes': ['Camiseta'],
    'Camisetas Series': ['Camiseta'],
    'Blusas Inverno': ['Blusa'],
    'Acessorios': ['Bolsa', 'Cinto', 'Óculos'],
    None: ['Camiseta', 'Blusa', 'Boné', 'Bolsa', 'Cinto', 'Óculos']}

def escolher_produto(campanha):
    produtos_possiveis = campanha_produto_relacao.get(campanha, campanha_produto_relacao[None])
    return np.random.choice(produtos_possiveis)

# Configuração para vendas
def generate_sales_data(crm_df, n=100):
        fake = Faker('pt_BR')
        random.seed(42)
        np.random.seed(42)

        meios_pagamento = ['Cartão', 'Pix', 'Boleto']
        status_pagamentos = ['Aprovado', 'Cancelado', 'Pendente']
        canais_origem = ['Instagram', 'Facebook', 'Google', 'Tráfego Direto', 'Email']
        produtos = ['Camiseta', 'Blusa', 'Boné', 'Bolsa', 'Cinto', 'Óculos']
        campaign_names = list(campanha_produto_relacao.keys())

        # Pegar apenas compradores
        compradores_df = crm_df[crm_df['etapa_funil'] == 'Comprou'].reset_index(drop=True)

        vendas_data = []

        # Define quem fará múltiplas compras (10% dos compradores)
        compradores_multiplas = compradores_df.sample(frac=0.1, random_state=42)

        # Garantir pelo menos uma venda para cada comprador
        for idx, row in compradores_df.iterrows():

                # Define o número de compras para esse lead
                num_compras = 2 if row['lead_id'] in compradores_multiplas['lead_id'].values else 1

                for _ in range(num_compras):
                        sale_id = fake.uuid4()
                        # Garante que a venda ocorra depois da data de captura
                        min_date = pd.to_datetime(row['data_captura']) + pd.Timedelta(days=1)
                        max_date = pd.to_datetime('today')
                        data_venda = fake.date_between(start_date=min_date, end_date=max_date)
                        
                        campanha = row['campanha_origem'] if row['campanha_origem'] in campanha_produto_relacao else None
                        produto = escolher_produto(campanha)  

                        # produto = np.random.choice(produtos, p=[0.5, 0.25, 0.1, 0.075, 0.025, 0.05])
                        valor_total = round(random.uniform(50, 500), 2)
                        quantidade = random.randint(1, 5)
                        meio_pagamento = np.random.choice(meios_pagamento, p=[0.7, 0.25, 0.05])
                        campanha_origem = np.random.choice(campaign_names, p=[0.45, 0.2, 0.15, 0.05, 0.15])
                        canal_origem =  row['canal_origem']

                        vendas_data.append({
                        "sale_id": sale_id,
                        "lead_id": row['lead_id'],
                        "nome_cliente": row['nome'],
                        "email_cliente": row['email'],
                        "data_venda": str(data_venda),
                        "produto": produto,
                        "valor_total": valor_total,
                        "quantidade": quantidade,
                        "meio_pagamento": meio_pagamento,
                        "status_pagamento": 'Aprovado',
                        "campanha_origem": campanha,
                        "utm_source": f"source_{random.randint(1, 10)}",
                        "utm_campaign": f"campaign_{random.randint(1, 5)}",
                        "canal_origem": canal_origem
                        })



# Gerar vendas aleatórias adicionais
        while len(vendas_data) < n:
                sale_id = fake.uuid4()
                data_venda = fake.date_between(start_date='-240d', end_date='today')
                campanha = np.random.choice(campaign_names)
                produto = escolher_produto(campanha)

                valor_total = round(random.uniform(50, 500), 2)
                quantidade = random.randint(1, 5)
                meio_pagamento = np.random.choice(meios_pagamento, p=[0.7, 0.25, 0.05])
                status_pagamento = np.random.choice(status_pagamentos, p=[0.65, 0.1, 0.25])
                canal_origem = np.random.choice(canais_origem, p=[0.3, 0.25, 0.25, 0.15, 0.05])

                # Se for aprovado, tenta usar um comprador real
                if status_pagamento == "Aprovado" and not compradores_df.empty:
                        comprador = compradores_df.sample(1).iloc[0]
                        nome_cliente = comprador['nome']
                        email_cliente = comprador['email']
                        lead_id = comprador['lead_id']
                        canal_origem = comprador['canal_origem']  # canal também vem do CRM
                else:
                        nome_cliente = fake.name()
                        nome_email = nome_cliente.lower().replace(' ', '.').replace('á','a').replace('ã','a').replace('â','a')\
                                                        .replace('é','e').replace('ê','e').replace('í','i').replace('ó','o')\
                                                        .replace('ô','o').replace('õ','o').replace('ú','u').replace('ç','c')
                        email_cliente = f"{nome_email}@email.com"
                        lead_id = None
                        canal_origem = np.random.choice(canais_origem, p=[0.3, 0.25, 0.25, 0.15, 0.05])

                vendas_data.append({
                "sale_id": sale_id,
                "lead_id": lead_id,
                "nome_cliente": nome_cliente,
                "email_cliente": email_cliente,
                "data_venda": str(data_venda),
                "produto": produto,
                "valor_total": valor_total,
                "quantidade": quantidade,
                "meio_pagamento": meio_pagamento,
                "status_pagamento": status_pagamento,
                "campanha_origem": campanha,
                "utm_source": f"source_{random.randint(1, 10)}",
                "utm_campaign": f"campaign_{random.randint(1, 5)}",
                "canal_origem": canal_origem
                })

        return pd.DataFrame(vendas_data)


In [7]:
vendas_df = generate_sales_data(crm_df, n=50)

# Descrição dos Dados

## 1. Data Shape

In [115]:
print(f'Numero de Linhas De cada Dataset:')
print(f'META ADS: {meta_df.shape[0]}')
print(f'CRM LEADS: {crm_df.shape[0]}')
print(f'VENDAS LOJA: {vendas_df.shape[0]}')

Numero de Linhas De cada Dataset:
META ADS: 1000
CRM LEADS: 300
VENDAS LOJA: 50


## 2. Data Types

In [126]:
print(meta_df.dtypes)

campanha                    object
conjunto_anuncio            object
anuncio                     object
impressoes                   int64
cliques                      int64
gasto_total                float64
ctr (%)                    float64
cpc (R$)                   float64
conversões                   int64
cpa (R$)                   float64
Receita                    float64
roas                       float64
data                datetime64[ns]
dia_da_semana               object
dtype: object


In [128]:
crm_df['ultima_interacao'] = pd.to_datetime(crm_df['ultima_interacao'])
print(crm_df.dtypes)

lead_id                     object
nome                        object
email                       object
telefone                    object
data_captura                object
campanha_origem             object
ad_clicked                  object
canal_origem                object
etapa_funil                 object
ultima_interacao    datetime64[ns]
status                      object
pontuacao                    int64
dtype: object


In [133]:
vendas_df['data_venda'] = pd.to_datetime(vendas_df['data_venda'])
print(vendas_df.dtypes)

sale_id                     object
lead_id                     object
nome_cliente                object
email_cliente               object
data_venda          datetime64[ns]
produto                     object
valor_total                float64
quantidade                   int64
meio_pagamento              object
status_pagamento            object
campanha_origem             object
utm_source                  object
utm_campaign                object
canal_origem                object
dtype: object


# Save Data

In [58]:
meta_df.to_csv("data/metaads_data.csv")
crm_df.to_csv("data/crm_data.csv")
vendas_df.to_csv("data/vendas_data.csv")

In [79]:
vendas_df_clean = vendas_df.drop(columns=['campanha_origem', 'canal_origem', 'email_cliente', 'nome_cliente'])
crm_sales = crm_df.merge(vendas_df_clean, on='lead_id', how='outer')


In [84]:
crm_sales.to_csv('data/crm_sales_data.csv')